In [14]:
import kfp
import kfp.dsl as dsl
from kfp.dsl import component, Input, Output, InputPath, OutputPath, Dataset, Metrics, Model, Artifact, ClassificationMetrics, Markdown

In [15]:
@component(
    packages_to_install=["torch","torchvision"]
)
def load_data(
    dataset: Output[Dataset]
):
    from torchvision import datasets
    from torchvision import transforms
    from torch.utils.data import DataLoader
    
    import os
    download_root = dataset.path # 데이터 다운로드 경로

    train_dataset = datasets.MNIST(root=download_root,
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True) # 학습 dataset 정의
                            
    test_dataset = datasets.MNIST(root=download_root,
                            train=False,
                            transform=transforms.ToTensor(), 
                            download=True) # 평가 dataset 정의
    

In [16]:
@dsl.pipeline(
    name="pytorch-v2"
)
def pipeline(
):
    load_data_task = load_data()

kfp.compiler.Compiler().compile(
pipeline_func=pipeline,
package_path='pytorch_v2.yaml')


In [17]:
import requests

USERNAME = "user@example.com"
PASSWORD = "12341234"
NAMESPACE = "kubeflow-user-example-com"
HOST = "http://172.17.0.11:8080" # istio-ingressgateway pod ip:port

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": "user@example.com", "password": "12341234"}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}",
)

In [18]:
arguments = {
}
client.create_run_from_pipeline_func(pipeline_func=pipeline,arguments=arguments,enable_caching=False)

RunPipelineResult(run_id=20de57ca-1529-4152-ab97-f733c1fff435)